In [1]:
# LINE GRAPH
def line_graph(data, choice, state, ylabel):
    data = data[data[choice] > 0]
    if data.empty == True:
        print("Unfortunately, this state has not provided enough information to create a graph")
    else:
        plt.plot(data.date, data[choice])
        plt.title(state + " - Line Graph")
        plt.xlabel("Date")
        plt.ylabel(ylabel)
        plt.show()

def line_graph_prediction(data, choice, state, ylabel, new_variable_dates, new_prediction):
    data = data[data[choice] > 0]
    if data.empty == True:
        print("Unfortunately, this state has not provided enough information to create a graph")
    else:
        plt.plot(data.date, data[choice])
        plt.plot(new_variable_dates[-93:], new_prediction[-93:,], linestyle='dashed', color="purple")
        plt.title(state + " - Line Graph")
        plt.xlabel("Date")
        plt.ylabel(ylabel)
        plt.show()
        
# BAR GRAPH
def bar_graph(data, choice, state, ylabel):
    data = data[data[choice] > 0]
    if data.empty == True:
        print("Unfortunately, this state has not provided enough information to create a graph")
    else:
        plt.bar(data.date, data[choice], align="center", alpha=0.5, color="g")
        plt.title(state + " - Bar Graph")
        plt.xlabel("Date")
        plt.ylabel(ylabel)
        plt.show()
        

def bar_graph_prediction(data, choice, state, ylabel, new_variable_dates, new_prediction):
    data = data[data[choice] > 0]
    if data.empty == True:
        print("Unfortunately, this state has not provided enough information to create a graph")
    else:
        plt.bar(data.date, data[choice], align="center", alpha=0.5, color="g")
        plt.bar(new_variable_dates[-93:], new_prediction[-93:,], align="center", alpha=0.5, color="purple")
        plt.title(state + " - Bar Graph")
        plt.xlabel("Date")
        plt.ylabel(ylabel)
        plt.show()
        
# PIE GRAPH
def pie_graph(data, state):
    selected_state_max = pd.DataFrame()
    indexes_to_drop = []
    state_df = pd.DataFrame([data.positive.max(), data.death.max(), data.hospitalized.max() ], index=['Positive', 'Deaths', 'Hospitalized'], columns=[state])
    selected_state_max = selected_state_max.append(state_df)
    
    for i in range(0, len(selected_state_max)):
        if pd.isna(selected_state_max.iloc[i].item()) or selected_state_max.iloc[i].item() <= 0:
            indexes_to_drop.append(i)
    
    selected_state_max.drop(selected_state_max.index[indexes_to_drop], inplace=True)
    
    
    if data.empty == True:
        print("Unfortunately, this state has not provided enough information to create a graph")
    else:
        selected_state_max.plot(kind='pie', title=state + " - Pie Chart", subplots=True, figsize=(12, 12), autopct='%1.1f%%')
        

def pie_graph_prediction(data, choice, state, new_prediction):
    selected_state_max = pd.DataFrame()
    indexes_to_drop = []
    state_df = pd.DataFrame([data.positive.max(), data.death.max(), data.hospitalized.max(), new_prediction.max()], index=['Positive', 'Deaths', 'Hospitalized', choice.capitalize() + ' Increase'], columns=[state])
    selected_state_max = selected_state_max.append(state_df)
    
    
    for i in range(0, len(selected_state_max)):
        if pd.isna(selected_state_max.iloc[i].item()) or selected_state_max.iloc[i].item() <= 0:
            indexes_to_drop.append(i)
    
    selected_state_max.drop(selected_state_max.index[indexes_to_drop], inplace=True)

    if data.empty == True:
        print("Unfortunately, this state has not provided enough information to create a graph")
    else:
        selected_state_max.plot(kind='pie', title=state + " - Pie Chart", subplots=True, figsize=(12, 12), autopct='%1.1f%%')

        
        


In [3]:
import pandas as pd
import datetime
from ipywidgets import widgets, interactive, Layout
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
import numpy as np 
import time
from sklearn.metrics import r2_score
from IPython.display import display
%matplotlib inline

covid_data_by_state = pd.read_csv('updated_data_by_state.csv')
plt.rcParams['figure.figsize'] = (19,6)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)



covid_data_by_state['date'] = pd.to_datetime(covid_data_by_state.date)

choose_state = widgets.Dropdown(
    options=list(covid_data_by_state['state'].unique()),
    description='State:',
    disabled=False
)

choose_option = widgets.Dropdown(
    options=["", "Deaths", "Hospitalized", "Positive"],
    description='Data:',
    disabled=False
)

prediction_check = widgets.Checkbox(
    value=False,
    description='Prediction',
    disabled=False,
    indent=False
)

show_data_check = widgets.Checkbox(
    value=False,
    description='Show Data',
    disabled=False,
    indent=False
)


def prediction_based_on_state(given_data, state, choice):
    data_for_prediction = given_data.sort_values(['date'], ascending=[1])
    days_to_predict = 90
    forecast = np.array([i for i in range(len(data_for_prediction['date'].unique())+days_to_predict)]).reshape(-1, 1)
    beginning = data_for_prediction.date.iloc[0].strftime("%m/%d/%Y")
    beginning_date = datetime.datetime.strptime(beginning, '%m/%d/%Y')
    forecast_dates = []
    for i in range(len(forecast)):
        forecast_dates.append((beginning_date + datetime.timedelta(days=i)).strftime('%m/%d/%Y'))


    days_since_january = np.array([i for i in range(len(data_for_prediction['date'].unique()))]).reshape(-1, 1)
    state_current_data = np.array(data_for_prediction[choice]).reshape(-1, 1)
    
    x_train, x_test, y_train, y_test = train_test_split(days_since_january[50:], state_current_data[50:], test_size=0.05)
    
    model = SVR(shrinking=True, kernel='poly')
    model.fit(x_train, y_train.ravel())
    accuracy = model.score(x_test, y_test).item() * 100.0
    print("Accuracy is: " + str(int(accuracy)) + "%")
    prediction = model.predict(forecast)
    

    added_dates = data_for_prediction.date   
    last_date = added_dates.iloc[len(data_for_prediction.date)-1]
    state_dates=[]
    for i in data_for_prediction.date:
        state_dates.append(i)

    for i in range(0, days_to_predict):
        state_dates.append(last_date + datetime.timedelta(days=i))

    
    result = [state_dates, prediction]

    return result        

        

def plot_graph_based_on_choice(state, option, prediction_box, show_data_box):
    prediction_check.layout.visibility= "hidden"
    data_copy = covid_data_by_state.copy()
    data_copy = data_copy[data_copy.state == state]
    selected_col = ""
    if option == "Deaths":
        selected_col = "death"
    if option == "Hospitalized":
        selected_col = "hospitalized"
    if option == "Positive":
        selected_col = "positive"


    output_for_data = widgets.Output(layout={'border': '1px solid black'})
    with output_for_data:
        if show_data_box == True:
            display(data_copy)
        else:
            pass
    display(output_for_data)
        
        
        
        
    
    if len(data_copy) > 0:
        if selected_col != "":
            data = data_copy[data_copy[selected_col] > 0]
            if data.empty == True:
                prediction_check.value=False
                prediction_check.layout.visibility="hidden"
                print("Unfortunately, this state has not provided enough information to create a graph")
            else:
                prediction_check.layout.visibility="visible"
                if prediction_box == False:
                    line_graph(data_copy, selected_col, state, option)
                    bar_graph(data_copy, selected_col, state, option)
                    pie_graph(data_copy, state)
                if prediction_box == True:    
                    if len(data) <= 65:
                        print("Unfortunately, there is not enough information to make an accurate prediction.")
                    else:
                        to_plot = prediction_based_on_state(data, state, selected_col)
                        line_graph_prediction(data_copy, selected_col, state, option, to_plot[0], to_plot[1])
                        bar_graph_prediction(data_copy, selected_col, state, option, to_plot[0], to_plot[1])
                        pie_graph_prediction(data_copy, selected_col, state, to_plot[1])
                        

In [4]:
application = interactive(plot_graph_based_on_choice, state=choose_state, option=choose_option, prediction_box=prediction_check, show_data_box=show_data_check)

In [6]:
from IPython.display import display
import getpass                                                                       
file=open("Credentials.txt","r")
lines=file.readlines()
lines = [x.strip() for x in lines]
stored_username=lines[0]
stored_password=lines[1]
file.close()

login = widgets.Button(description="Login")
output = widgets.Output()
exit = widgets.Button(description="Exit")

username_field = widgets.Text(
    value='',
    placeholder='Username',
    description='Username:',
    disabled=False
)

password_field = widgets.Password(
    value='',
    placeholder='Enter password',
    description='Password:',
    disabled=False
)


username_field.layout.visibility="visible"
password_field.layout.visibility="visible"
login.layout.visibility="visible"
output.layout.visibility="visible"
application.layout.visibility="hidden"
exit.layout.visibility="hidden"
prediction_check.layout.visibility= "hidden"

display(username_field, password_field, login, output, exit)

def on_login_clicked(b):
    with output:
        while True:
            username = username_field.value
            password = password_field.value
            if username == stored_username and password == stored_password:
                print("welcome!")
                username_field.layout.visibility="hidden"
                password_field.layout.visibility="hidden"
                login.layout.visibility="hidden"
                output.layout.visibility="hidden"
                application.layout.visibility="visible"
                exit.layout.visibility="visible"
                prediction_check.layout.visibility= "visible"
                break
            else:
                print("Incorrect Username or Password")
                break

def on_exit_clicked(b):
    username_field.value = ""
    password_field.value = ""
    output.clear_output()
    username_field.layout.visibility="visible"
    password_field.layout.visibility="visible"
    login.layout.visibility="visible"
    output.layout.visibility="visible"
    application.layout.visibility="hidden"
    exit.layout.visibility="hidden"
    prediction_check.layout.visibility= "hidden"

login.on_click(on_login_clicked)
exit.on_click(on_exit_clicked)
application

Text(value='', description='Username:', layout=Layout(visibility='visible'), placeholder='Username')

Password(description='Password:', layout=Layout(visibility='visible'), placeholder='Enter password')

Button(description='Login', layout=Layout(visibility='visible'), style=ButtonStyle())

Output(layout=Layout(visibility='visible'))

Button(description='Exit', layout=Layout(visibility='hidden'), style=ButtonStyle())

interactive(children=(Dropdown(description='State:', options=('AK', 'AL', 'AR', 'AS', 'AZ', 'CA', 'CO', 'CT', …